In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style="ticks", color_codes=True)
plt.style.use('ggplot')
import researchpy as rp
from scipy import stats
%matplotlib inline
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from statsmodels.discrete.discrete_model import Logit
from statsmodels.tools import add_constant


In [6]:
df = pd.read_csv('data/clean_df')
df1 = pd.read_csv('data/clean_df1')

/Users/claudiostaub/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df1.drop(['Unnamed: 0'], axis=1, inplace=True)

In [11]:
X = np.array(df[['A1Cresult', 'num_medications', 'time_in_hospital', 'num_lab_procedures',
       'num_procedures', 'number_inpatient']].values)



In [12]:
X = np.array(df[['num_medications', 'time_in_hospital', 'num_lab_procedures',
       'num_procedures', 'number_inpatient']].values)
y = np.array(df['readmitted_<30d'])

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state=0)
logit = sm.Logit(y_train, X_train)
result = logit.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.385502
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                76521
Model:                          Logit   Df Residuals:                    76516
Method:                           MLE   Df Model:                            4
Date:                Sun, 30 Jun 2019   Pseudo R-squ.:                -0.08855
Time:                        21:38:17   Log-Likelihood:                -29499.
converged:                       True   LL-Null:                       -27099.
                                        LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.0557      0.002    -33.959      0.000      -0.059      -0.053
x2            -0.0135      0.

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [14]:
clf = RandomForestClassifier(n_estimators=100, random_state=0, oob_score=True)
clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=True, random_state=0, verbose=0, warm_start=False)

In [15]:
clf.feature_importances_

array([0.27266418, 0.11977215, 0.46558271, 0.08360281, 0.05837816])

In [16]:
clf.predict(X_test)

array([False, False, False, ..., False, False, False])

In [17]:
clf.score(X_test,y_test)

0.8646699074800063

In [18]:
clf.oob_score_
# 100 estimators

0.8596071666601325

In [19]:
dummytest = df1[[ 'encounter_id', 'patient_nbr', 'race', 'gender', 'age',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'number_diagnoses',
       'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
       'miglitol', 'troglitazone', 'tolazamide', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted_<30d']]

In [31]:
dummytest = pd.get_dummies(dummytest, drop_first=True)

### Dummy variables 

Lets test the model with dummy variables. We will remove the diagnosis columns from this as this creates a very large amount of columns. 

In [33]:
X1 = np.array(dummytest.drop(['readmitted_<30d'], axis=1).values)
y1 = np.array(dummytest['readmitted_<30d'].values)

X_traindummy, X_testdummy, y_traindummy, y_testdummy = train_test_split(X1,y1, test_size=0.20, random_state=0)
clf1 = RandomForestClassifier(n_estimators=100,
                              max_features='auto',
                              random_state=0, oob_score=True)
clf1.fit(X_traindummy, y_traindummy)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=True, random_state=0, verbose=0, warm_start=False)

In [34]:
clf1.oob_score_
## 1000 estimators with X1 (dummies) and y1 (dummies) as input

0.8870196751617588

In [35]:
clf1.score(X_testdummy, y_testdummy)

0.8839592246342365

Comparing the two feature sets, the dummy set seemed to perform better. Let's do some further testing by varying the amount of estimators. 

In [36]:
n_estimators = [10,50,100,500]

In [37]:
def random_forest_est(X, y, n_estimators ):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.2, random_state=0)
    result = [['n_estimators', 'oob_score', 'test_score']]
    for n in n_estimators:
        rf = RandomForestClassifier(n_estimators=n, max_features='auto',
                                   random_state=0, oob_score=True)
        rf.fit(X_train, y_train)
        result.append([n, rf.oob_score_, rf.score(X_test,y_test)])
    return result
    

In [39]:
random_forest_est(X1, y1, n_estimators)

/Users/claudiostaub/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/claudiostaub/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:463: RuntimeWarning: divide by zero encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/claudiostaub/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:463: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


[['n_estimators', 'oob_score', 'test_score'],
 [10, 0.8666710682688499, 0.8816352400570433],
 [50, 0.8862934108015318, 0.8838535889716368],
 [100, 0.8870196751617588, 0.8839592246342365],
 [500, 0.8872177472600027, 0.8838535889716368]]

In [48]:
test_size = [0.1, 0.15, 0.2]
def random_forest_test_size(X, y, test_size ):
    result = [['test_size', 'oob_score', 'test_score']]
    for size in test_size:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=size, random_state=0)
        rf = RandomForestClassifier(n_estimators=100, max_features='auto',
                                   random_state=0, oob_score=True)
        rf.fit(X_train, y_train)
        result.append([size, rf.oob_score_, rf.score(X_test,y_test)])
    return result

In [49]:
random_forest_test_size(X1, y1, test_size)

[['test_size', 'oob_score', 'test_score'],
 [0.1, 0.8867082961641392, 0.8844406887081441],
 [0.15, 0.8865814100891093, 0.8848591549295775],
 [0.2, 0.8870196751617588, 0.8839592246342365]]